In [1]:
# Importing Libraries

import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import preprocessing
import seaborn as sns
from matplotlib import pyplot as plt
#import pandas_profiling

In [2]:
## Importing 2021 Datasets

data_1 = pd.read_csv('2021VAERSDATA.csv',encoding='ISO-8859-1')
data_2_symptoms = pd.read_csv('2021VAERSSYMPTOMS.csv',encoding='ISO-8859-1')
data_3_vax = pd.read_csv('2021VAERSVAX.csv',encoding='ISO-8859-1')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7,12,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_3_vax.head(10)

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME
0,916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
1,916601,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA))
2,916602,COVID19,PFIZER\BIONTECH,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH))
3,916603,COVID19,MODERNA,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA))
4,916604,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
5,916605,FLUC4,"SEQIRUS, INC.",276563,1,SYR,LA,INFLUENZA (SEASONAL) (FLUCELVAX QUADRIVALENT)
6,916606,COVID19,MODERNA,011J20A,1,IM,LA,COVID19 (COVID19 (MODERNA))
7,916607,COVID19,MODERNA,NaN,UNK,IM,LA,COVID19 (COVID19 (MODERNA))
8,916608,COVID19,MODERNA,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA))
9,916609,COVID19,MODERNA,011J201A,1,IM,LA,COVID19 (COVID19 (MODERNA))


In [4]:
data_3_vax.shape

(505121, 8)

In [5]:
## Filtering the Covid Data

data_3_vax = data_3_vax[data_3_vax['VAX_TYPE'] == 'COVID19']

In [6]:
data_3_vax.shape

(494251, 8)

In [42]:
data_2_symptoms.head(10)

,VAERS_ID,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916600,Dysphagia,23.1,Epiglottitis,23.1,NaN,NaN,NaN,NaN,NaN,NaN
1,916601,Anxiety,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916602,Chest discomfort,23.1,Dysphagia,23.1,Pain in extremity,23.1,Visual impairment,23.1,NaN,NaN
3,916603,Dizziness,23.1,Fatigue,23.1,Mobility decreased,23.1,NaN,NaN,NaN,NaN
4,916604,Injection site erythema,23.1,Injection site pruritus,23.1,Injection site swelling,23.1,Injection site warmth,23.1,NaN,NaN
5,916605,Chills,24.0,Confusional state,24.0,Eye inflammation,24.0,Headache,24.0,Laboratory test,24.0
6,916605,Pyrexia,24.0,White blood cell count decreased,24.0,NaN,NaN,NaN,NaN,NaN,NaN
7,916606,Pharyngeal swelling,23.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,916607,Abdominal pain,23.1,Chills,23.1,Sleep disorder,23.1,NaN,NaN,NaN,NaN
9,916608,Diarrhoea,23.1,Nasal congestion,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data_2_symptoms['SYMPTOM2'].unique()

array(['Epiglottitis', 'Dyspnoea', 'Dysphagia', ...,
       'Epidural haemorrhage', 'Mononeuropathy', 'Pelvic prolapse'],
      dtype=object)

In [9]:
#Joining Symptoms and Vax files

data_symp_vax = pd.merge(data_3_vax,data_2_symptoms,how = 'left', on=['VAERS_ID'] )

In [10]:
data_symp_vax.head(2)

,VAERS_ID,VAX_TYPE,VAX_MANU,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916600,COVID19,MODERNA,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA)),Dysphagia,23.1,Epiglottitis,23.1,NaN,NaN,NaN,NaN,NaN,NaN
1,916601,COVID19,MODERNA,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA)),Anxiety,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
data_symp_vax.shape

(664812, 18)

In [12]:
data_symp_vax['VAX_MANU'].unique()

array(['MODERNA', 'PFIZER\\BIONTECH', 'UNKNOWN MANUFACTURER', 'JANSSEN'],
      dtype=object)

In [13]:
## Filtering out the Unknown Manufacturer

data_symp_vax = data_symp_vax[data_symp_vax['VAX_MANU'] != 'UNKNOWN MANUFACTURER']

In [14]:
data_symp_vax.columns

Index(['VAERS_ID', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME', 'SYMPTOM1', 'SYMPTOMVERSION1',
       'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3',
       'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5'],
      dtype='object')

In [15]:
# Exporting the Combined File
data_symp_vax.to_csv('data_symp_vax.csv')

In [16]:
data_symp_vax_1 = data_symp_vax.drop(['VAX_TYPE', 'VAX_NAME','SYMPTOM1', 'SYMPTOMVERSION1',
                                      'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3',
                                      'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5',
                                      'VAX_LOT', 'VAX_SITE', 'VAX_ROUTE'], axis = 1)

In [43]:
data_symp_vax_1.head(10)

,VAERS_ID,VAX_MANU,VAX_DOSE_SERIES
0,916600,MODERNA,1
1,916601,MODERNA,1
2,916602,PFIZER\BIONTECH,1
3,916603,MODERNA,UNK
4,916604,MODERNA,1
5,916606,MODERNA,1
6,916607,MODERNA,UNK
7,916608,MODERNA,1
8,916609,MODERNA,1
9,916610,MODERNA,1


In [18]:
data_symp_vax_1['VAX_DOSE_SERIES'].unique()

array(['1', 'UNK', '2', '3', '5', nan, '4', '7+', '6'], dtype=object)

In [19]:
data_symp_vax_1.shape

(663320, 3)

In [20]:
data_symp_vax_1.drop_duplicates(subset ="VAERS_ID", keep = False, inplace = True)

In [21]:
data_symp_vax_1.shape

(343491, 3)

In [22]:
data_1.head(2)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,None,None,NaN,NaN,2,01/01/2021,NaN,Y,NaN,Pcn and bee venom
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Patient residing at nursing facility. See pati...,Patient residing at nursing facility. See pati...,NaN,NaN,2,01/01/2021,NaN,Y,NaN,"""Dairy"""


In [23]:
data_1.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES'],
      dtype='object')

In [24]:
## Removing unwanted Columns

data_2 = data_1.drop(['SYMPTOM_TEXT', 'CUR_ILL', 'HISTORY', 'ALLERGIES','OTHER_MEDS','SYMPTOM_TEXT'], axis = 1)

In [25]:
data_2.shape

(484964, 30)

In [26]:
data_2.drop_duplicates(subset ="VAERS_ID", keep = False, inplace = True)

In [27]:
data_2.shape

(484964, 30)

In [28]:
#Final Data, joining with Symp and Vax File

data_comb = pd.merge(data_2, data_symp_vax_1,how = 'left', on=['VAERS_ID'], )

In [44]:
data_comb.head(10)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,DIED,DATEDIED,...,V_FUNDBY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,VAX_MANU,VAX_DOSE_SERIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,Y,NaN,MODERNA,1
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,Y,NaN,MODERNA,1
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,Y,PFIZER\BIONTECH,1
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,NaN,NaN,...,NaN,"got measles from measel shot, mums from mumps ...",NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,UNK
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,1
5,916605,01/01/2021,TX,40.0,40.0,NaN,M,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,Y,NaN,0,NaN
6,916606,01/01/2021,NV,44.0,44.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,1
7,916607,01/01/2021,KS,50.0,50.0,NaN,M,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,UNK
8,916608,01/01/2021,OH,33.0,33.0,NaN,M,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,1
9,916609,01/01/2021,TN,71.0,71.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,NaN,NaN,MODERNA,1


In [30]:
data_comb.shape

(484964, 32)

In [31]:
data_comb['VAX_MANU'].unique()

array(['MODERNA', 'PFIZER\\BIONTECH', nan, 'JANSSEN'], dtype=object)

In [32]:
data_comb['VAX_MANU'].fillna(0, inplace=True)

In [33]:
data_comb.shape

(484964, 32)

In [34]:
## Filtering out NA Manufacturers

data_comb_1 = data_comb[data_comb['VAX_MANU'] != 0]

In [35]:
data_comb_1.shape

(343491, 32)

In [36]:
data_comb_1.head(2)

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,DIED,DATEDIED,...,V_FUNDBY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,VAX_MANU,VAX_DOSE_SERIES
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,Y,NaN,MODERNA,1
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,NaN,NaN,...,NaN,NaN,NaN,2,01/01/2021,NaN,Y,NaN,MODERNA,1


In [37]:
## Removing unwanted Columns

data_comb_2 = data_comb_1.drop(['VAERS_ID', 'CAGE_YR', 'CAGE_MO', 'RPT_DATE', 'VAX_DATE',
                               'TODAYS_DATE', 'RECVDATE', 'RPT_DATE', 'ONSET_DATE', 'LAB_DATA', 'DATEDIED', 'VAX_DOSE_SERIES',
                               'FORM_VERS', 'PRIOR_VAX', 'SPLTTYPE' ], axis = 1)

In [38]:
## Replacing Y with 1

data_comb_2['DIED'] = data_comb_2['DIED'].str.replace('Y', '1')
data_comb_2['L_THREAT'] = data_comb_2['L_THREAT'].str.replace('Y', '1')
data_comb_2['ER_VISIT'] = data_comb_2['ER_VISIT'].str.replace('Y', '1')
data_comb_2['HOSPITAL'] = data_comb_2['HOSPITAL'].str.replace('Y', '1')
data_comb_2['X_STAY'] = data_comb_2['X_STAY'].str.replace('Y', '1')
data_comb_2['DISABLE'] = data_comb_2['DISABLE'].str.replace('Y', '1')
data_comb_2['RECOVD'] = data_comb_2['RECOVD'].str.replace('Y', '1')
data_comb_2['RECOVD'] = data_comb_2['RECOVD'].str.replace('N', '0')
data_comb_2['RECOVD'] = data_comb_2['RECOVD'].str.replace('U', '2')
data_comb_2['BIRTH_DEFECT'] = data_comb_2['BIRTH_DEFECT'].str.replace('Y', '1')
data_comb_2['ER_ED_VISIT'] = data_comb_2['ER_ED_VISIT'].str.replace('Y', '1')
data_comb_2['OFC_VISIT'] = data_comb_2['OFC_VISIT'].str.replace('Y', '1')
data_comb_2['BIRTH_DEFECT'] = data_comb_2['BIRTH_DEFECT'].str.replace('Y', '1')

In [39]:
## Replacing Missing Values with Zero

data_comb_2.fillna(0, inplace=True)

In [45]:
data_comb_2.head(10)

,STATE,AGE_YRS,SEX,DIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,NUMDAYS,V_ADMINBY,V_FUNDBY,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,VAX_MANU
0,TX,33.0,F,0,0,0,0,0.0,0,0,1,2.0,PVT,0,0,1,0,MODERNA
1,CA,73.0,F,0,0,0,0,0.0,0,0,1,0.0,SEN,0,0,1,0,MODERNA
2,WA,23.0,F,0,0,0,0,0.0,0,0,2,0.0,SEN,0,0,0,1,PFIZER\BIONTECH
3,WA,58.0,F,0,0,0,0,0.0,0,0,1,0.0,WRK,0,0,0,0,MODERNA
4,TX,47.0,F,0,0,0,0,0.0,0,0,0,7.0,PUB,0,0,0,0,MODERNA
6,NV,44.0,F,0,0,0,0,0.0,0,0,1,0.0,PVT,0,0,0,0,MODERNA
7,KS,50.0,M,0,0,0,0,0.0,0,0,1,1.0,PUB,0,0,0,0,MODERNA
8,OH,33.0,M,0,0,0,0,0.0,0,0,0,2.0,OTH,0,0,0,0,MODERNA
9,TN,71.0,F,0,0,0,0,0.0,0,0,0,8.0,PUB,0,0,0,0,MODERNA
10,VA,18.0,F,0,0,0,0,0.0,0,0,0,1.0,PVT,0,0,0,0,MODERNA


In [41]:
data_comb_2.to_csv('Analysis_ready.csv')